In [1]:
#encoding=utf-8
import jieba
import numpy as np
import time
from collections import Counter
import sys
import math
import csv
doc_count = 0
dict_size = 0
word_count = 0
vocabulary = []
vocabulary_dict = {}
punct_list = []
docs = []
raw_docs = []
doc_metric_matrix = []
known_words = []
comprehensible_collection = []
frequency_dictionary = {}
comp_max = 0.97
comp_min = 0.75
input_file = "wiki-zh.txt"

#seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
#print("Full Mode: " + "/ ".join(seg_list))  # 全模式


#seg_list = jieba.cut(text, cut_all=False)
#print("Default Mode: " + "/ ".join(seg_list))  # 默认模式

#seg_list = jieba.cut("他来到了网易杭研大厦")
#print(", ".join(seg_list))

#seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
#print(", ".join(seg_list))

In [2]:
def build_chinese_punct_list():
    p_list = []
    with open("input/punct.txt", encoding="utf8") as f:
        for line in f:
            line = line.strip('\n')
            p_list.append(line)
    p_list.append('\n')
    p_list.append('\t')
    return p_list

In [3]:
def strip_chinese_punctuation(str):
    str2 = str.strip('\n')
    str2 = str2.translate({ord(i):' ' for i in punct_list})
    return str2

In [4]:
def build_corpus():
        seg_list = []
        word_list = []
        documents = []
        r_docs = []
        d_count = 0
        w_count = 0
        file = open(input_file, encoding="utf8")
        N = 50000
        for i in range(N):
            #print(line)
            r_docs.append(file.readline())
            line = strip_chinese_punctuation(r_docs[i])
            word_list = []
            seg_list = jieba.cut(line, cut_all=False)
            #print("Segmentation: " + "/ ".join(seg_list))
            for x in seg_list:
                if x != ' ':
                    word_list.append(x)
                    w_count += 1
            documents.append(word_list)
            d_count+=1
        file.close()
        return r_docs, documents, d_count, w_count

In [5]:
def initialize_doc_metric_matrix():
    doc_count = len(docs)
    #print(doc_count)
    doc_mm = np.zeros((doc_count, 6))
    for i in range(0, doc_count):
        doc_mm[i][0] = i
    return doc_mm
    
#doc_metric_matrix = initialize_doc_metric_matrix()
#print(doc_metric_matrix)

In [6]:
def build_vocabulary():
        v_size = 0
        v_dict = {}
        vocab = list(set(x for l in docs for x in l))
        for i in vocab:
            v_dict.update({i:False})
        v_size = len(vocab)
        #print(vocabulary_size)
        return vocab, v_size, v_dict

In [7]:
def collect_known_words():
    with open("input/known_vocab.txt", encoding="utf8") as f:
        for line in f:
            line = line.strip('\n')
            vocabulary_dict[line] = True
    
    #print(known_list)
        #for i in vocabulary:
            #known[col] = sum(1 for k in )

In [8]:
def get_doc_word_counts():
    for i in range(0,doc_count):
        doc_metric_matrix[i][1] = len(docs[i])

In [9]:
def build_collection_freq_list():
    freq_dict = {"first":0}
    for i in vocabulary:
        #print(i)
        freq_dict.update({i:0})
        
    for i in range(0,doc_count):
        for j in docs[i]:
            freq_dict[j]+=1
    return freq_dict

In [10]:
def get_doc_known_count():
    known_counter = 0
    known_bool = False
    freq_sum = 0.0
    
    for h in range(0, doc_count):
        doc_word_index = Counter(docs[h])
        #print(doc_word_index[known_words[0]])
        known_counter = 0
        freq_sum = 0.0
        for i in doc_word_index:
            known_bool = False
            if vocabulary_dict[i] == True:
                known_counter += doc_word_index[i]
                known_bool = True
            
            #for j in known_words:
                #print(i + " " + j + '\n')
                #if i == j:
                    #known_counter += doc_word_index[i]
                    #known_bool = True
                    
            if known_bool == False:
                freq_sum += (-1)/math.log(frequency_dictionary[i]/word_count)
                
        doc_metric_matrix[h][2] = known_counter
        doc_metric_matrix[h][4] = 100*freq_sum/doc_metric_matrix[h][1]
        



In [11]:
def calculate_known_ratio():
    for i in range(0,doc_count):
        doc_metric_matrix[i][3] = doc_metric_matrix[i][2] / doc_metric_matrix[i][1]
    
    #doc_metric_matrix = doc_metric_matrix[np.argsort(cc[:,3])]


In [12]:
def get_comprehensible_collection():
    cc = doc_metric_matrix[((doc_metric_matrix[:,3] > comp_min) & (doc_metric_matrix[:,3] <= comp_max))]
    cc = cc[np.argsort(cc[:,3])]
    return cc

In [13]:
def print_freq_lists_to_file():
    with open('output/freq_list_all.csv', 'w',encoding='utf8',newline="") as csv_file:
        writer = csv.writer(csv_file)
        for key, value in frequency_dictionary.items():
            writer.writerow([key, value])
    
    with open('output/freq_list_repeated_words.csv', 'w',encoding='utf8',newline="") as csv_file:
        writer = csv.writer(csv_file)
        for key, value in frequency_dictionary.items():
            if value > 1:
                writer.writerow([key, value])

In [14]:
def print_comprehensible_collection_to_file():
    with open('output/comprehensible_collection.txt', 'w', encoding='utf8') as txt_writer:
        writer = txt_writer
        for i in range(0, len(comprehensible_collection)):
            writer.write(raw_docs[int(comprehensible_collection[i][0])])
    np.savetxt('output/comprehensible_collection_metrics.csv', comprehensible_collection, delimiter=',', header="doc_id,word_count,known_word_count,known_ratio,unknown_c_score,blank", fmt=['%d','%d','%d','%f','%f','%f'])        

In [15]:
punct_list = build_chinese_punct_list()
raw_docs, docs, doc_count, word_count = build_corpus()
vocabulary, dict_size, vocabulary_dict = build_vocabulary()
known_words = collect_known_words()
doc_metric_matrix = initialize_doc_metric_matrix()
get_doc_word_counts()
frequency_dictionary = build_collection_freq_list()
get_doc_known_count()
calculate_known_ratio()
comprehensible_collection = get_comprehensible_collection()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.640 seconds.
Prefix dict has been built succesfully.


In [16]:
print_freq_lists_to_file()
print_comprehensible_collection_to_file()

In [17]:
#from wiki_dump_reader import Cleaner, iterate

#cleaner = Cleaner()
#for title, text in iterate(r'C:\Users\Curt\Downloads\zhwiki-20191101-pages-articles-multistream.xml\zhwiki-20191101-pages-articles-multistream.xml'):
#    text = cleaner.clean_text(text)

In [18]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
#comp_min = 0.5
comprehensible_collection = get_comprehensible_collection()
print(comprehensible_collection)

[[11121.      445.      343.        0.771     1.593     0.   ]
 [27738.      329.      255.        0.775     1.709     0.   ]
 [15293.      783.      618.        0.789     1.438     0.   ]
 [ 6323.      231.      184.        0.797     1.612     0.   ]]


In [19]:
print(doc_metric_matrix)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
from operator import itemgetter
sorted(frequency_dictionary.items(), key=itemgetter(1), reverse=True)

[('的', 1727176),
 ('在', 477987),
 ('是', 350138),
 ('年', 294213),
 ('和', 246553),
 ('了', 236294),
 ('於', 215161),
 ('為', 202573),
 ('有', 189117),
 ('後', 145918),
 ('中', 142333),
 ('被', 138937),
 ('與', 135718),
 ('也', 134958),
 ('他', 130857),
 ('为', 119871),
 ('等', 108894),
 ('人', 101999),
 ('而', 101443),
 ('之', 94159),
 ('由', 90270),
 ('上', 89898),
 ('以', 86047),
 ('但', 85814),
 ('日', 82629),
 ('香港', 80950),
 ('zh', 73087),
 ('及', 71564),
 ('与', 71431),
 ('月', 69909),
 ('到', 67742),
 ('不', 66001),
 ('都', 65576),
 ('日本', 60838),
 ('臺', 58398),
 ('the', 57810),
 ('其', 57518),
 ('對', 54955),
 ('軍', 54464),
 ('至', 52758),
 ('后', 50279),
 ('會', 49256),
 ('中国', 48808),
 ('或', 47955),
 ('of', 47612),
 ('來', 47206),
 ('使用', 46038),
 ('又', 45494),
 ('大', 44900),
 ('可以', 44160),
 ('于', 44130),
 ('就', 43999),
 ('包括', 43660),
 ('站', 42816),
 ('并', 42690),
 ('第', 42367),
 ('長', 41944),
 ('馬', 40836),
 ('新', 39273),
 ('陳', 39158),
 ('主要', 38913),
 ('其中', 38037),
 ('世界', 37310),
 ('我', 37275),
 ('時', 

In [21]:
word_count

45645598

In [22]:
dict_size

2081230